In [ ]:
# !python -> colab default, !python3.5 -> 3.5.10
# It takes up to 15 minutes.

# Install Python 3.5
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y build-essential libssl-dev libffi-dev 
!apt-get install -y python3.5 python3.5-dev
!wget https://bootstrap.pypa.io/pip/3.5/get-pip.py && python3.5 get-pip.py
!rm -rf /content/get-pip.py


# Install Cuda 9.0
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!rm -rf /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb


# Install hent-AI
!git clone https://github.com/kdrkdrkdr/GoodbyeLaver
%cd /content/GoodbyeLaver/hent-AI/
!wget https://github.com/kdrkdrkdr/GoodbyeLaver/releases/download/1.0/4x_FatalPixels_340000_G.pth
!wget https://github.com/kdrkdrkdr/GoodbyeLaver/releases/download/1.0/weights.h5
!python3.5 -m pip install -r requirements-gpu.txt


# Install lama_cleaner
%cd /content/GoodbyeLaver/lama_cleaner/
!python -m pip install -r requirements.txt


# Make directory
!mkdir /content/image_original/
!mkdir /content/image_detected/
!mkdir /content/image_uncensored/


# Run lama server
%cd /content/GoodbyeLaver/
!nohup python lama.py --port 5003 > log_lama.out &

In [ ]:
# Run Decensoring!
# Upload the censored image to the "/content/image_original/" folder, and run this cell.
!rm -rf /content/image_detected/*
!rm -rf /content/image_uncensored/*

# Detecting bar censored
%cd /content/GoodbyeLaver/hent-AI/
!python3.5 run.py

# Inpainting image
%cd /content/GoodbyeLaver/
!python3 inpainting.py

In [6]:
# Run this cell if you want to delete all uploaded images.
!rm -rf /content/image_original/*